# Jigsaw selection #1

A selection from [`attempt_01.ipynb`](./attempt_01.ipynb) and [`attempt_02.ipynb`](./attempt_02.ipynb) to be ordered to manufacturer.

In [1]:
from collections import Counter

import numpy as np

from tqdm import tqdm

from jigsaw import *
from jigsaw.default import *
from jigsaw.svg import *

In [2]:
def is_unique(horizontal_edges, vertical_edges, *, constraints="border"):
    H, _ = horizontal_edges.shape
    _, W = vertical_edges.shape
    pieces = grid_to_pieces(horizontal_edges, vertical_edges, opposite)
    return has_unique_solution(H, W, pieces, opposite, flip, constraints=constraints)


def search(sampler, *validators, max_iterations=None):

    # Loop until a grid with a unique solution is found
    i = 0
    with tqdm(total=max_iterations) as progress:
        while max_iterations is None or i < max_iterations:

            # Sample grid, according to rules
            horizontal_edges, vertical_edges = sampler()
            
            # Ask validators for compliance
            try:
                for validator in validators:
                    if not validator(horizontal_edges, vertical_edges):
                        break
                else:
                    return horizontal_edges, vertical_edges
            finally:
                progress.update(1)
            
            i += 1

In [3]:
def count_solutions(horizontal_edges, vertical_edges, *, constraints="border"):
    H, _ = horizontal_edges.shape
    _, W = vertical_edges.shape
    pieces = grid_to_pieces(horizontal_edges, vertical_edges, opposite)
    iterator = iterate_unique_solutions(H, W, pieces, opposite, flip, constraints=constraints)
    count = sum(1 for _ in iterator)
    return count

def display_solutions(horizontal_edges, vertical_edges, *, constraints="border"):
    H, _ = horizontal_edges.shape
    _, W = vertical_edges.shape
    pieces = grid_to_pieces(horizontal_edges, vertical_edges, opposite)
    for h, v in iterate_unique_solutions(H, W, pieces, opposite, flip, constraints=constraints):
        display_grid(h, v)

In [4]:
def has_unique_pieces(horizontal_edges, vertical_edges):
    pieces = grid_to_pieces(horizontal_edges, vertical_edges, opposite)
    pieces = canonize_piece(pieces, flip)
    pieces = pieces.reshape(-1, 4)
    unique_pieces = np.unique(pieces, axis=0)
    return pieces.shape[0] == unique_pieces.shape[0]

In [5]:
def is_inner_unique(horizontal_edges, vertical_edges):
    inner_horizontal_edges = horizontal_edges[1:-1, 1:-1]
    inner_vertical_edges = vertical_edges[1:-1, 1:-1]
    horizontal_constraints = np.full_like(inner_horizontal_edges, -1)
    vertical_constraints = np.full_like(inner_vertical_edges, -1)
    constraints = horizontal_constraints, vertical_constraints
    return is_unique(inner_horizontal_edges, inner_vertical_edges, constraints=constraints)

## 1. 4x4 with 5th corner (with constraints)

In [6]:
# This will be a 4x4 grid, with an additional corner
H, W = 4, 4
horizontal_edges, vertical_edges = sample_random_grid(H, W, np.array([FLAT_ALTERNATIVE]))
horizontal_edges[1, 2] = FLAT
vertical_edges[2, 1] = FLAT

# First, make sure that the overall layout can only be solved one way
assert is_unique(horizontal_edges, vertical_edges)

# Show it, to illustrate the purpose
display_grid(horizontal_edges, vertical_edges)

In [7]:
# Generate grid
H, W = 4, 4
generator = np.random.default_rng(42)
def sampler():
    
    # Generate a grid with small edges
    edge_types = np.array([
        CENTERED_SMALL_MALE,
        CENTERED_SMALL_FEMALE,
    ])
    h_small, v_small = sample_random_grid(H, W, edge_types, generator=generator)

    # Generate a grid with medium edges
    edge_types = np.array([
        CENTERED_MEDIUM_MALE,
        CENTERED_MEDIUM_FEMALE,
    ])
    h_medium, v_medium = sample_random_grid(H, W, edge_types, generator=generator)

    # Mix, with a bias in favor of small edges
    horizontal_edges = np.where(generator.random(h_small.shape) < 0.8, h_small, h_medium)
    vertical_edges = np.where(generator.random(v_small.shape) < 0.8, v_small, v_medium)
    
    # Add 5th corner
    # Note: there is only one way to place it (x4, due to rotation)
    horizontal_edges[1, 2] = FLAT
    vertical_edges[2, 1] = FLAT
    
    # Make sure that the 5th corner has only female edges
    horizontal_edges[1, 1] = CENTERED_SMALL_FEMALE
    vertical_edges[1, 1] = CENTERED_SMALL_MALE
    
    # Make sure that the two extra edge pieces have female edges on the side
    horizontal_edges[2, 1] = CENTERED_SMALL_FEMALE
    horizontal_edges[2, 2] = CENTERED_SMALL_MALE
    vertical_edges[1, 2] = CENTERED_SMALL_MALE
    vertical_edges[2, 2] = CENTERED_SMALL_FEMALE
    
    return horizontal_edges, vertical_edges

horizontal_edges, vertical_edges = search(sampler, is_unique)

171it [00:01, 111.08it/s]


In [8]:
# Show it
display_grid(horizontal_edges, vertical_edges)

In [9]:
# Keep it for later
pieces_4x4_5th = grid_to_pieces(horizontal_edges, vertical_edges, opposite)

## 2. 4x4 split

In [10]:
# Generate grid
H, W = 4, 4
generator = np.random.default_rng(42)
def sampler():
    
    # Start from a simple grid, with small edges
    edge_types = np.array([
        CENTERED_SMALL_MALE,
        CENTERED_SMALL_FEMALE,
        LEFT_SMALL_MALE,
        LEFT_SMALL_FEMALE,
        RIGHT_SMALL_MALE,
        RIGHT_SMALL_FEMALE,
    ])
    horizontal_edges, vertical_edges = sample_random_grid(H, W, edge_types, generator=generator)
    
    # Cut staircase
    for i in range(1, H):
        horizontal_edges[i, i] = FLAT
        vertical_edges[i, i - 1] = FLAT
    
    # Replace lower half with medium centered edges
    edge_types = np.array([
        CENTERED_MEDIUM_MALE,
        CENTERED_MEDIUM_FEMALE,
    ])
    for i in range(1, H - 1):
        horizontal_edges[i+1, 1:i+1] = generator.choice(edge_types, i)
        vertical_edges[i+1:H, i-1] = generator.choice(edge_types, H - 1 - i)
    
    return horizontal_edges, vertical_edges
    
horizontal_edges, vertical_edges = search(sampler, is_unique)

1it [00:00, 50.02it/s]


In [11]:
# Show it
display_grid(horizontal_edges, vertical_edges)

In [12]:
# Keep it for later
pieces_4x4_split = grid_to_pieces(horizontal_edges, vertical_edges, opposite)

## 3. 4x4 snake

_Updated to have unique pieces._

In [13]:
# Generate grid
generator = np.random.default_rng(42)
def sampler():
    
    # Start from a simple grid, with small edges
    edge_types = np.array([
        CENTERED_SMALL_MALE,
        LEFT_SMALL_MALE,
        DOUBLE_SMALL_MALE,
        TWISTED_SMALL_MALE,
    ])
    edge_types = expand_edge_types(edge_types, opposite, flip)
    horizontal_edges, vertical_edges = sample_random_grid(4, 4, edge_types, generator=generator)
    
    # Draw snake outline
    horizontal_edges[0, 2] = FLAT
    horizontal_edges[1, 2] = FLAT
    horizontal_edges[1, 3] = FLAT
    horizontal_edges[2, 1] = FLAT
    horizontal_edges[2, 3] = FLAT
    vertical_edges[1, 0] = FLAT
    vertical_edges[2, 1] = FLAT
    vertical_edges[3, 1] = FLAT
    vertical_edges[3, 2] = FLAT
    
    return horizontal_edges, vertical_edges
    
horizontal_edges, vertical_edges = search(sampler, has_unique_pieces, is_unique)

789it [00:04, 168.12it/s]


In [14]:
# Show it
display_grid(horizontal_edges, vertical_edges)

In [15]:
# Keep it for later
pieces_4x4_snake = grid_to_pieces(horizontal_edges, vertical_edges, opposite)

## 4. 4x4 with pure square

_This is an updated version, which add unicity of solution of the inner pieces._

In [16]:
# This will be a 4x4 grid
H, W = 4, 4
horizontal_edges, vertical_edges = sample_random_grid(H, W, np.array([FLAT_ALTERNATIVE]))
horizontal_edges[0, 1] = FLAT
vertical_edges[1, 0] = FLAT

# First, make sure that the overall layout can only be solved one way only
assert is_unique(horizontal_edges, vertical_edges)

# Show it, to illustrate the purpose
display_grid(horizontal_edges, vertical_edges)

In [17]:
# Choose edge types
edge_types = np.array([
    CENTERED_SMALL_MALE,
    #LEFT_SMALL_MALE,
    CENTERED_MEDIUM_MALE,
])
edge_types = expand_edge_types(edge_types, opposite, flip)

# Generate grid
generator = np.random.default_rng(123123)
def sampler():
    horizontal_edges, vertical_edges = sample_random_grid(H, W, edge_types, generator=generator)
    
    # Add square in upper-left corner
    horizontal_edges[0, 1] = FLAT
    vertical_edges[1, 0] = FLAT
    
    # Central pieces have male pins "radiating" outside
    horizontal_edges[1:3, 1] = generator.choice([CENTERED_SMALL_MALE, CENTERED_MEDIUM_MALE], size=2)
    horizontal_edges[1:3, 3] = generator.choice([CENTERED_SMALL_FEMALE, CENTERED_MEDIUM_FEMALE], size=2)
    vertical_edges[1, 1:3] = generator.choice([CENTERED_SMALL_FEMALE, CENTERED_MEDIUM_FEMALE], size=2)
    vertical_edges[3, 1:3] = generator.choice([CENTERED_SMALL_MALE, CENTERED_MEDIUM_MALE], size=2)

    # Corners that are not next to the square are male only (i.e. they cannot connect with the radiating center)
    horizontal_edges[[0, 3], 3] = generator.choice([CENTERED_SMALL_MALE, CENTERED_MEDIUM_MALE], size=2)
    horizontal_edges[3, 1] = generator.choice([CENTERED_SMALL_FEMALE, CENTERED_MEDIUM_FEMALE])
    vertical_edges[3, [0, 3]] = generator.choice([CENTERED_SMALL_FEMALE, CENTERED_MEDIUM_FEMALE], size=2)
    vertical_edges[1, 3] = generator.choice([CENTERED_SMALL_MALE, CENTERED_MEDIUM_MALE])
    
    return horizontal_edges, vertical_edges

horizontal_edges, vertical_edges = search(sampler, has_unique_pieces, is_inner_unique, is_unique)

15140it [00:21, 703.85it/s]


In [18]:
# Show it
display_grid(horizontal_edges, vertical_edges)

In [19]:
# Keep it for later
pieces_4x4_pure = grid_to_pieces(horizontal_edges, vertical_edges, opposite)

## 5. 4x4 with more edges

In [20]:
# This will be a 4x4 grid
H, W = 4, 4
horizontal_edges, vertical_edges = sample_random_grid(H, W, np.array([FLAT_ALTERNATIVE]))
vertical_edges[2, 1:-1] = FLAT

# First, make sure that the overall layout can only be solved one way
assert is_unique(horizontal_edges, vertical_edges)

# Show it, to illustrate the purpose
display_grid(horizontal_edges, vertical_edges)

In [21]:
# Choose edge types
edge_types = np.array([
    CENTERED_SMALL_MALE,
    LEFT_SMALL_MALE,
])

# Automatically expand types to match combinations
edge_types = expand_edge_types(edge_types, opposite, flip)

# Show actual types
edge_names[edge_types]

array(['CENTERED_SMALL_MALE', 'CENTERED_SMALL_FEMALE', 'RIGHT_SMALL_MALE',
       'RIGHT_SMALL_FEMALE', 'LEFT_SMALL_MALE', 'LEFT_SMALL_FEMALE'],
      dtype='<U22')

In [22]:
# Generate grid
H, W = 4, 4
generator = np.random.default_rng(78125)
def sampler():
    horizontal_edges, vertical_edges = sample_random_grid(H, W, edge_types, generator=generator)
    
    # Add an horizontal split, so that the inner pieces are also edge pieces
    vertical_edges[2, 1:-1] = FLAT
    
    # Force the two "internal" edge pairs to have double pins
    horizontal_edges[1:-1, 2] = DOUBLE_SMALL_MALE
    
    # Force one end of these "pairs" to have female centered pins
    horizontal_edges[1:-1, 1] = CENTERED_SMALL_FEMALE
    
    # And then force corners to not provide any male centered pins
    # Therefore, these "pairs" cannot be connected to a corner
    horizontal_edges[[0, 3], 1] = generator.choice([CENTERED_SMALL_MALE, LEFT_SMALL_MALE, LEFT_SMALL_FEMALE, RIGHT_SMALL_MALE, RIGHT_SMALL_FEMALE], size=2)
    horizontal_edges[[0, 3], 3] = generator.choice([CENTERED_SMALL_FEMALE, LEFT_SMALL_MALE, LEFT_SMALL_FEMALE, RIGHT_SMALL_MALE, RIGHT_SMALL_FEMALE], size=2)
    vertical_edges[1, [0, 3]] = generator.choice([CENTERED_SMALL_FEMALE, LEFT_SMALL_MALE, LEFT_SMALL_FEMALE, RIGHT_SMALL_MALE, RIGHT_SMALL_FEMALE], size=2)
    vertical_edges[3, [0, 3]] = generator.choice([CENTERED_SMALL_MALE, LEFT_SMALL_MALE, LEFT_SMALL_FEMALE, RIGHT_SMALL_MALE, RIGHT_SMALL_FEMALE], size=2)
    
    return horizontal_edges, vertical_edges
horizontal_edges, vertical_edges = search(sampler, has_unique_pieces, is_unique)

6it [00:00, 111.11it/s]


In [23]:
# Show it
display_grid(horizontal_edges, vertical_edges)

In [24]:
# Keep it for later
pieces_4x4_edge = grid_to_pieces(horizontal_edges, vertical_edges, opposite)

## 6. 4x4 with 6 corners

In [25]:
# This will be a 4x4 grid
H, W = 4, 4
horizontal_edges, vertical_edges = sample_random_grid(H, W, np.array([FLAT_ALTERNATIVE]))
horizontal_edges[0, 2] = FLAT

# First, make sure that the overall layout can only be solved one way
assert is_unique(horizontal_edges, vertical_edges)

# Show it, to illustrate the purpose
display_grid(horizontal_edges, vertical_edges)

In [26]:
# Choose edge types
edge_types = np.array([
    CENTERED_SMALL_MALE,
    LEFT_SMALL_MALE,
])

# Automatically expand types to match combinations
edge_types = expand_edge_types(edge_types, opposite, flip)

# Show actual types
edge_names[edge_types]

array(['CENTERED_SMALL_MALE', 'CENTERED_SMALL_FEMALE', 'RIGHT_SMALL_MALE',
       'RIGHT_SMALL_FEMALE', 'LEFT_SMALL_MALE', 'LEFT_SMALL_FEMALE'],
      dtype='<U22')

In [27]:
# Generate grid
H, W = 4, 4
generator = np.random.default_rng(78125)
def sampler():
    horizontal_edges, vertical_edges = sample_random_grid(H, W, edge_types, generator=generator)
    
    # Add a single flat edge, so that the top edge pieces become corners
    horizontal_edges[0, 2] = FLAT
    
    # Add some double pins, in such a way that there cannot be ambiguity
    # Pairs can be easily identified (but not necessarily orientation)
    horizontal_edges[3, 1] = DOUBLE_SMALL_MALE
    vertical_edges[1, 1] = DOUBLE_SMALL_MALE
    vertical_edges[3, 2] = DOUBLE_SMALL_MALE
    
    return horizontal_edges, vertical_edges
horizontal_edges, vertical_edges = search(sampler, has_unique_pieces, is_unique)

3it [00:00, 136.41it/s]


In [28]:
# Show it
display_grid(horizontal_edges, vertical_edges)

In [29]:
# Keep it for later
pieces_4x4_6_corners = grid_to_pieces(horizontal_edges, vertical_edges, opposite)

## 7. 5x5 with nested corners

In [30]:
# This will be a 5x5 grid, with 4 additional corned inside
H, W = 5, 5
horizontal_edges, vertical_edges = sample_random_grid(H, W, np.array([FLAT_ALTERNATIVE]))
horizontal_edges[[1, 1, 3, 3], [1, 4, 1, 4]] = FLAT
vertical_edges[[1, 1, 4, 4], [1, 3, 1, 3]] = FLAT

# First, make sure that the overall layout can only be solved one way
assert is_unique(horizontal_edges, vertical_edges)

# Show it, to illustrate the purpose
display_grid(horizontal_edges, vertical_edges)

In [31]:
# Outer perimeter cannot contain any centered small pin
# This makes it easier to identify which corners are inside
outer_edge_types = np.array([
    LEFT_SMALL_MALE,
    LEFT_SMALL_FEMALE,
    RIGHT_SMALL_MALE,
    RIGHT_SMALL_FEMALE,
] * 2 + [
    DOUBLE_SMALL_MALE,
    DOUBLE_SMALL_FEMALE,
    TWISTED_SMALL_MALE,
    TWISTED_SMALL_FEMALE,
])

generator = np.random.default_rng(13746)

def sampler():
    
    # Start with empty (i.e. flat-only) 5x5 grid
    H = W = 5
    horizontal_edges = np.zeros((H, W + 1), dtype=np.uint8)
    vertical_edges = np.zeros((H + 1, W), dtype=np.uint8)
    
    # Sample outer perimeter
    horizontal_edges[[0, 0, 0, 0, 4, 4, 4, 4], [1, 2, 3, 4, 1, 2, 3, 4]] = generator.choice(outer_edge_types, size=8)
    vertical_edges[[1, 2, 3, 4, 1, 2, 3, 4], [0, 0, 0, 0, 4, 4, 4, 4]] = generator.choice(outer_edge_types, size=8)
    
    # Inner area is set to wildcard, for the moment
    horizontal_edges[[1, 1, 2, 2, 2, 2, 3, 3], [2, 3, 1, 2, 3, 4, 2, 3]] = FLAT_ALTERNATIVE
    vertical_edges[[2, 3, 1, 2, 3, 4, 2, 3], [1, 1, 2, 2, 2, 2, 3, 3]] = FLAT_ALTERNATIVE
    
    return horizontal_edges, vertical_edges

outer_horizontal_edges, outer_vertical_edges = search(sampler, is_unique)

234it [00:06, 35.18it/s]


In [32]:
# Show it
display_grid(outer_horizontal_edges, outer_vertical_edges)

In [33]:
inner_edge_types = np.array([
    CENTERED_SMALL_MALE,
    CENTERED_SMALL_FEMALE,
    LEFT_SMALL_MALE,
    LEFT_SMALL_FEMALE,
    RIGHT_SMALL_MALE,
    RIGHT_SMALL_FEMALE,
] * 4 + [
    DOUBLE_SMALL_MALE,
    DOUBLE_SMALL_FEMALE,
    #TWISTED_SMALL_MALE,
    #TWISTED_SMALL_FEMALE,
])

generator = np.random.default_rng(123)

def sampler():
    
    # Start with selected solution
    horizontal_edges = outer_horizontal_edges.copy()
    vertical_edges = outer_vertical_edges.copy()
    
    # Inner can contain twisted pins as well
    horizontal_edges[[1, 1, 2, 2, 2, 2, 3, 3], [2, 3, 1, 2, 3, 4, 2, 3]] = generator.choice(inner_edge_types, size=8)
    vertical_edges[[2, 3, 1, 2, 3, 4, 2, 3], [1, 1, 2, 2, 2, 2, 3, 3]] = generator.choice(inner_edge_types, size=8)
    
    # Place centered ones manually, to ensure diversity
    horizontal_edges[1, 2] = CENTERED_SMALL_MALE
    horizontal_edges[3, 2] = CENTERED_SMALL_MALE
    horizontal_edges[3, 3] = CENTERED_SMALL_MALE
    vertical_edges[2, 3] = CENTERED_SMALL_FEMALE
    vertical_edges[3, 1] = CENTERED_SMALL_FEMALE
    vertical_edges[1, 2] = CENTERED_SMALL_FEMALE
    
    return horizontal_edges, vertical_edges

horizontal_edges, vertical_edges = search(sampler, is_unique)

8it [00:08,  1.03s/it]


In [34]:
# Show it
display_grid(horizontal_edges, vertical_edges)

In [35]:
# Focus on the inner part
inner_horizontal_edges = horizontal_edges[1:-1, 1:-1]
inner_vertical_edges = vertical_edges[1:-1, 1:-1]

# Show it
display_grid(inner_horizontal_edges, inner_vertical_edges)

In [36]:
# Checking unicity on this subset is a bit more tricky
# We need to allow any pins on the central edges
horizontal_constraints = np.full((3, 3 + 1), -1)
vertical_constraints = np.full((3 + 1, 3), -1)
horizontal_constraints[0, [0, -1]] = FLAT
horizontal_constraints[2, [0, -1]] = FLAT
vertical_constraints[[0, -1], 0] = FLAT
vertical_constraints[[0, -1], 2] = FLAT
constraints = horizontal_constraints, vertical_constraints

# Hopefully there is only a unique solution
count_solutions(inner_horizontal_edges, inner_vertical_edges, constraints=constraints)

1

In [37]:
# Keep it for later
pieces_5x5_nested = grid_to_pieces(horizontal_edges, vertical_edges, opposite)

## 8. 5x5 with 5th corner

In [38]:
# Generate grid
generator = np.random.default_rng(456)
def sampler():
    
    # Generate a grid with small edges
    edge_types = np.array([
        CENTERED_SMALL_MALE,
        CENTERED_SMALL_FEMALE,
    ] * 3 + [
        LEFT_SMALL_MALE,
        LEFT_SMALL_FEMALE,
        RIGHT_SMALL_MALE,
        RIGHT_SMALL_FEMALE,
    ] * 2 + [
        DOUBLE_SMALL_MALE,
        DOUBLE_SMALL_FEMALE,
        TWISTED_SMALL_MALE,
        TWISTED_SMALL_FEMALE,
    ])
    horizontal_edges, vertical_edges = sample_random_grid(5, 5, edge_types, generator=generator)

    # Add 5th corner
    # Note: there is only one way to place it (x4, due to rotation)
    horizontal_edges[2, 3] = FLAT
    vertical_edges[3, 2] = FLAT
    
    # Make sure that the 5th corner has only female edges
    horizontal_edges[2, 2] = CENTERED_SMALL_FEMALE
    vertical_edges[2, 2] = CENTERED_SMALL_MALE
    
    # Make sure that the two extra edge pieces have femal edges on the side
    horizontal_edges[3, 2] = CENTERED_SMALL_FEMALE
    horizontal_edges[3, 3] = CENTERED_SMALL_MALE
    vertical_edges[2, 3] = CENTERED_SMALL_MALE
    vertical_edges[3, 3] = CENTERED_SMALL_FEMALE

    # Make sure that the two extra edge pieces have double edges (one of each polarity)
    horizontal_edges[2, 4] = DOUBLE_SMALL_MALE
    vertical_edges[4, 2] = DOUBLE_SMALL_MALE
    
    return horizontal_edges, vertical_edges

def is_acceptable(horizontal_edges, vertical_edges):
    
    # Make sure no other corner has only centered small female edges
    if horizontal_edges[0, 1] == CENTERED_SMALL_MALE and vertical_edges[1, 0] == CENTERED_SMALL_FEMALE:
        return False
    if horizontal_edges[0, 4] == CENTERED_SMALL_FEMALE and vertical_edges[1, 4] == CENTERED_SMALL_FEMALE:
        return False
    if horizontal_edges[4, 1] == CENTERED_SMALL_MALE and vertical_edges[4, 0] == CENTERED_SMALL_MALE:
        return False
    if horizontal_edges[4, 4] == CENTERED_SMALL_FEMALE and vertical_edges[4, 4] == CENTERED_SMALL_MALE:
        return False

    # Make sure no other edge piece has two small female on the sides
    for i in range(4):
        for j in [0, 4]:
            if horizontal_edges[j, i+1] == CENTERED_SMALL_FEMALE and horizontal_edges[j, i+2] == CENTERED_SMALL_MALE:
                return False
            if vertical_edges[i+1, j] == CENTERED_SMALL_MALE and vertical_edges[i+2, j] == CENTERED_SMALL_FEMALE:
                return False
    
    # Make sure no other edge piece has double pins on the inner side
    if (
        horizontal_edges[1, 1] in {DOUBLE_SMALL_MALE, DOUBLE_SMALL_FEMALE} or
        horizontal_edges[2, 1] in {DOUBLE_SMALL_MALE, DOUBLE_SMALL_FEMALE} or
        horizontal_edges[3, 1] in {DOUBLE_SMALL_MALE, DOUBLE_SMALL_FEMALE} or
        horizontal_edges[1, 4] in {DOUBLE_SMALL_MALE, DOUBLE_SMALL_FEMALE} or
        horizontal_edges[3, 4] in {DOUBLE_SMALL_MALE, DOUBLE_SMALL_FEMALE} or
        vertical_edges[1, 1] in {DOUBLE_SMALL_MALE, DOUBLE_SMALL_FEMALE} or
        vertical_edges[1, 2] in {DOUBLE_SMALL_MALE, DOUBLE_SMALL_FEMALE} or
        vertical_edges[1, 3] in {DOUBLE_SMALL_MALE, DOUBLE_SMALL_FEMALE} or
        vertical_edges[4, 1] in {DOUBLE_SMALL_MALE, DOUBLE_SMALL_FEMALE} or
        vertical_edges[4, 3] in {DOUBLE_SMALL_MALE, DOUBLE_SMALL_FEMALE}
    ):
        return False

    return True

horizontal_edges, vertical_edges = search(sampler, is_acceptable, is_unique)

9it [00:00, 163.67it/s]


In [39]:
# Show it
display_grid(horizontal_edges, vertical_edges)

In [40]:
# Keep it for later
pieces_5x5_5th = grid_to_pieces(horizontal_edges, vertical_edges, opposite)

## Export as SVG

In [41]:
# Pack jigsaw puzzles as a single grid
pieces = np.zeros((8, 12, 4), dtype=np.uint8)
pieces[0:4, 0:4] = pieces_4x4_5th
pieces[4:8, 0:4] = pieces_4x4_split
pieces[0:4, 4:8] = pieces_4x4_snake
pieces[4:8, 4:8] = pieces_4x4_pure
pieces[0:4, 8:12] = pieces_4x4_edge
pieces[4:8, 8:12] = pieces_4x4_6_corners
horizontal_edges, vertical_edges = pieces_to_grid(pieces, opposite)
display_grid(horizontal_edges, vertical_edges, scale=50.0, margin=0)
with open("selection_01.svg", "w") as file:
    file.write(grid_to_svg(horizontal_edges, vertical_edges, scale=25.0, margin=1/25))

In [42]:
# TODO pieces_5x5_nested, pieces_5x5_5th

## Individual packing

In [43]:
from jigsaw.svg import get_style
from IPython.core.display import HTML, display

C:\Users\Jojo\AppData\Local\Temp\ipykernel_13952\2873478938.py:2: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import HTML, display


In [44]:
scale = 25.0
style = get_style()

origin = np.array([0.0, 0.0])
ex = np.array([scale, 0.0])
ey = np.array([0.0, scale])

data = []

h, w, _ = pieces.shape
for i in range(h):
    for j in range(w):
        piece = pieces[i, j, :]
        o = origin + (i * 2 + 0.5) * ey + (j * 2 + 0.5) * ex

        data.append("M {} {}".format(*o))
        style.trace_edge(data, piece[2], o, ey, -ex)
        style.trace_edge(data, piece[3], o + ey, ex, ey)
        style.trace_edge(data, piece[0], o + ex + ey, -ey, ex)
        style.trace_edge(data, piece[1], o + ex, -ex, -ey)
        data.append("Z")

data = " ".join(data)

path = f'<path fill="none" stroke="black" d="{data}"/>'
height = int(scale) * h * 2 + 1
width = int(scale) * w * 2 + 1
svg = f'<svg width="{width}" height="{height}" xmlns="http://www.w3.org/2000/svg">{path}</svg>'

with open("selection_01_split.svg", "w") as file:
    file.write(svg)

widget = HTML(f'<div style="padding:1em">{svg}</div>')
display(widget)